# 이전까지 배웠던 것들

In [1]:
import datasets
import transformers
import evaluate
import numpy as np
import torch # 파이토치

In [3]:
train_dict = {
    'text': [
        "I prefer making decisions based on logic and objective facts.",
        "I always consider how others might feel when making a decision.",
        "Data and analysis drive most of my decisions.",
        "I rely on my empathy and personal values to guide my choices."
    ],
    'label': [0, 1, 0, 1]  # 0은 T(사고형), 1은 F(감정형)
}

test_dict = {
    'text': [
        "I find it important to weigh all the pros and cons logically.",
        "When making decisions, I prioritize harmony and people's emotions."
    ],
    'label': [0, 1]  # 0은 T(사고형), 1은 F(감정형)
}

In [6]:
train_data = datasets.Dataset.from_dict(train_dict)
test_data = datasets.Dataset.from_dict(test_dict)
나의데이터 = datasets.dataset_dict.DatasetDict({'train':train_data, 'test':test_data})

In [7]:
나의데이터

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2
    })
})

In [8]:
데이터전처리하기1 = 토크나이저 = transformers.AutoTokenizer.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\qnsgh\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\qnsgh\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

C:\Users\qnsgh\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [32]:
인공지능 = model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased",
    num_labels=2 )
# ^^^ 인공지능 생성기


인공지능.classifier.weight
# ^^^ 인공지능의 대량의 숫자들(tensor)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameter containing:
tensor([[ 0.0032, -0.0022, -0.0045,  ...,  0.0322,  0.0115, -0.0124],
        [ 0.0261,  0.0465,  0.0261,  ...,  0.0176,  0.0243, -0.0202]],
       requires_grad=True)

`-` 인공지능: 인공지능의 파라메터는 변화할 수 있으며, loss가 더 작은쪽으로 파라메터를 변화시키는 과정을 “학습”이라고 부른다.

`-` 인공지능: 인공지능은 “`**Dict 
 transformers.modeling_outputs.SequenceClassifierOutput”`인 함수이다. 그런데 쓰기 까다롭다.

- `1.` `Dict`에는 특정한 key를 포함하고 있어야한다. (input_ids, attention_mask)
- `2.` `key`에 대응하는 숫자는 !파이토치 텐서!형태이어야 한다.

```Python
    np.array([1,2,3]) VS. torch.tensor([1,2,3])
   ```
^^^ 인공지능을 학습시키기 위해 GPU가 필수인데, 후자의 경우가 GPU에 특화된 입력 형태 $\to$ (`3.` 따라서 (m,n)꼴의 차원을 반드시 가져야 한다. 값이 존재하지 않으면 0으로 메꾸는 형태...Truncation, Padding etc)
- `4.` `Dict`에 `labels`이 (텐서형으로) 포함된 경우 loss가 계산된다. (그리고 이걸 계산해야지 학습을 할 수 있음)

In [27]:
토크나이저(나의데이터['train']['text'][0])

{'input_ids': [101, 1045, 9544, 2437, 6567, 2241, 2006, 7961, 1998, 7863, 8866, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [28]:
인공지능입력1 = {
        'input_ids': torch.tensor([[ 101, 1045, 102],[101, 9544, 102]]), 
        'attention_mask': torch.tensor([[1, 1, 1],[1, 1, 1]]) # 생략가능
}

# ^^^ 2번 설명 자료

In [29]:
궁금 = 토크나이저(나의데이터['train']['text'][0])
type(궁금['input_ids'])

list

In [33]:
인공지능(**인공지능입력1)

SequenceClassifierOutput(loss=None, logits=tensor([[0.0399, 0.0468],
        [0.0289, 0.0282]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

여기서 `인공지능입력1`이란 단순히 `torch.tensor(input_ids)` 의 값을 의미하며(<1>`input_ids` 말고 `attention_mask`도 올 수 있음. 후자의 경우는 선택임. 

<2> `torch.tensor(input_ids)`는 인공지능의 확신 정도(?)를 의미함.), 함수 `인공지능(**인공지능입력1)` 은 인공지능 model의 logits 값을 출력함.

여기서는 `labels` 가 없기 때문에 이 인공지능이 정상적으로 수행하는지 검토하는 사용으로 활용됨. (=학습 목적이 아님.)


In [38]:
인공지능입력2 = {
        'input_ids': torch.tensor([[ 101, 1045, 102],[101, 9544, 102]]), 
        'attention_mask': torch.tensor([[1, 1, 1],[1, 1, 1]]), # 생략가능
        'labels': torch.tensor([1, 0]) # 생략가능
}

In [39]:
인공지능(**인공지능입력2)

SequenceClassifierOutput(loss=tensor(0.6913, grad_fn=<NllLossBackward0>), logits=tensor([[0.0399, 0.0468],
        [0.0289, 0.0282]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

**인공지능입력1과 달라진 것들** :

1. 입력값에 `labels`가 추가됨: 음... `labels`는 마치 수학 문제집의 정답지 역할을 함. 인공지능입력1이 그저 문풀만 했다면(∵ `labels`가 없어서), 인공지능입력2는 `labels`가 추가됨에 따라 인공지능이 스스로 학습할 수 있음.

2. loss의 값이 등장함: `labels`가 새롭게 추가 되었기 때문.(정답지를 보고 자신의 점수(=`loss`)를 확인할 수 있음.

In [54]:
인공지능출력 = 인공지능(**인공지능입력2)
로짓 = 인공지능출력.logits
로짓

tensor([[0.0399, 0.0468],
        [0.0289, 0.0282]], grad_fn=<AddmmBackward0>)

In [55]:
torch.exp(로짓) / torch.exp(로짓).sum(axis=1)

tensor([[0.4983, 0.5092],
        [0.4928, 0.4998]], grad_fn=<DivBackward0>)

sum(axis=1)이란 한 행을 모두 더하라는 걸로 알고 있음. (추후 최규빈 교수님의 파이썬 프로그래밍 강좌를 다시 보자.)

그래서, 

```Python
torch.exp(로짓) / torch.exp(로짓).sum(axis=1)
```
로짓 = [0.0399,0.0468] 일 때, 그에 대한 exp 값은

[exp(0.0399)/{exp(0.0399) + exp(0.0468)} , exp(0.0468)/{exp(0.0399) + exp(0.0468)} ] 이 됨.

In [56]:
def 확률계산하기(인공지능출력):
    로짓 = 인공지능출력.logits
    return torch.exp(로짓) / torch.exp(로짓).sum(axis=1)

In [57]:
확률계산하기(인공지능(**인공지능입력2))

tensor([[0.4983, 0.5092],
        [0.4928, 0.4998]], grad_fn=<DivBackward0>)

당연히 확률이므로, 합하면 1이 됨. 이 프로그램은 T/F를 찾는 인공지능을 만들고 있으며 각각 0, 1에 대응함.

확률을 보면 거의 반반이라는 것을 알 수 있는데, 이는 인공지능이 아주 아주 멍청한 상태(∵ 학습을 하지 않았기 때문.)라는 것을 알 수 있음.

"그냥 아무거나 찔러본다" 라고 생각하자.

In [49]:
전처리된나의데이터 = 나의데이터.map(lambda dct: 토크나이저(dct["text"], truncation=True))

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

`map()`함수는 <1> function을 입력값으로 받음.
```Python
나의데이터.map(lambda dct: 토크나이저(dct["text"], truncation=True))
```
가 가능한 이유.

<2> function 중에서도 `dict`를 받음. dict라고 한다면 `DatasetDict`가 생각나야 할 것이다!
 

## 데이터콜렉터 (더 전처리 된 데이터)

목적: 인공지능이 학습하기 쉽게 행렬의 형태로 만들기. tokenizer의 전처리 이후 실행하므로 숫자화(O), datacollator로 인해 행렬화한다.ㅡ

`torch.tensor`, `미니배치`, `패딩/동적패딩` 모두 O.  (당연함. 인공지능의 학습을 위해 구조를 떠먹어줘야 하기 때문.)

datacollator를 위해 `dictionary`의 형태로 바꿔줘야 함.(`label`, `text`, `input_ids`, `attention_mask`)


# 평가하기 (accuracy)

In [60]:
accuracy = evaluate.load("accuracy")

In [61]:
accuracy.compute(predictions=[0,0,0], references=[0,0,0])

{'accuracy': 1.0}

In [62]:
accuracy.compute(predictions=[1,0,0], references=[0,0,0])

{'accuracy': 0.6666666666666666}

# to("cuda:0")

```Python
확률계산하기(인공지능(**데이터콜렉터([토크나이저("This was a masterpiece.")]).to("cuda:0")))
```

`.to("cuda:0")` 는 해당 데이터를 GPU 상에 두었다는 것을 의미함.

만약 데이터 간 위치(GPU vs. CPU) 가 다르다면 코드가 돌아가지 않음. 하나로 통일해줘야 함.
